# 작업 1유형

### 1-1
결측치가 하나라도 존재하는 행의 경우 경우 해당 행을 삭제하라. 그후 남은 데이터의 상위 70%에 해당하는 데이터만 남겨둔 후 median_income 컬럼의 1분위수를 반올림하여 소숫점이하 2째자리까지 구하여라

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p1_1.csv')
df.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [6]:
df = df.dropna().reset_index(drop=True)
df_filter = df.iloc[:int(len(df)*0.7)]
print(round(df_filter.median_income.quantile(q=0.25),2))

2.51


### 1-2
1990년도는 해당년도 평균 이하 GDP를 가지지만, 2010년도에는 해당년도 평균 이상 GDP를 가지는 국가의 숫자를 구하여라

In [7]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p1_2.csv')
df.head()

,Country Name,Country Code,Year,Value
0,Arab World,ARB,1968,2576068.3
1,Arab World,ARB,1969,2843420.4
2,Arab World,ARB,1970,3138550.0
3,Arab World,ARB,1971,3642691.0
4,Arab World,ARB,1972,4331605.7


In [28]:
df_1990 = df[df.Year ==1990]
df_2010 = df[df.Year ==2010]


df_1990_filter = df_1990[df_1990.Value <= df_1990.Value.mean()]
df_2010_filter = df_2010[df_2010.Value >= df_2010.Value.mean()]

result = len(set(df_2010_filter['Country Code']) & set(df_1990_filter['Country Code']))
print(result)

5


### 1-3
데이터에서 결측치가 가장 많은 컬럼을 출력하라



In [29]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p1_3.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,NaN,S
1,2,1,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,C85,C
2,3,1,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,C85,S
3,4,1,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S
4,5,0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0.0,373450,8.0500,C123,S


In [40]:
result = df.isna().sum().to_frame('na')
print(result.sort_values('na').index[-1])

Fare


# 작업 2유형
TravelInsurance가 1일 확률을 구해서 제출하라. 평가지표 : auc
<br>제출 파일 컬럼 : ID, proba

In [41]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p2_train_.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p2_test_.csv')
train.head()

,ID,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,1008,26,Private Sector/Self Employed,Yes,1400000,5,0,No,Yes,1
1,199,30,Private Sector/Self Employed,No,1450000,5,0,Yes,Yes,1
2,86,32,Government Sector,Yes,900000,4,0,No,No,0
3,560,26,Private Sector/Self Employed,Yes,1400000,7,0,No,Yes,1
4,161,34,Private Sector/Self Employed,No,1400000,3,1,No,Yes,1


In [45]:
# print(train.isna().sum(), test.isna().sum())  # 결측치 없음
# print(train.info(), test.info())  
# object : Employment Type, GraduateOrNot, FrequentFlyer, EverTravelledAbroad

y = train['TravelInsurance']
train_d = train.drop(columns=['ID','TravelInsurance'])
test_d = test.drop(columns='ID')

train_dd = pd.get_dummies(train_d)
test_dd = pd.get_dummies(test_d)
test_dd = test_dd[train_dd.columns]

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

X_train, X_val, y_train, y_val = train_test_split(train_dd, y, test_size=0.3, stratify=y, random_state=0)
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)

## 예측, 성능평가
val_proba = rfc.predict_proba(X_val)[:,1]
proba = rfc.predict_proba(test_dd)[:,1]

print('roc_auc_score', roc_auc_score(y_val, val_proba))
pd.DataFrame({'ID':test.ID, 'proba':proba}).to_csv('results/test3_result.csv', index=False)

roc_auc_score 0.8004782494758911


# 작업 3유형
투약 후 체중에서 투약 전 체중을 뺏을 때 값은 일반 적으로 세가지 등급으로 나눈다. <br>
-3이하 : A등급, -3초과 0이하 : B등급, 0 초과 : C등급. 
<br>약 실험에서 A,B,C 그룹간의 인원 수 비율은 2:1:1로 알려져 있다. 
<br>위 데이터 표본은 각 범주의 비율에 적합한지 카이제곱 검정하려한다.

In [1]:
import pandas as pd 
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p3_1.csv')
df.head()

,ID,투약전,투약후
0,user_1,55.137,56.106
1,user_2,66.584,60.409
2,user_3,52.259,50.639
3,user_4,77.081,69.164
4,user_5,62.677,57.622


### 3-1-a
A등급에 해당하는 유저는 몇명인지 확인하라

In [7]:
lst = []
for x in range(0,len(df.ID)) :
    if (df.투약후[x] - df.투약전[x]) <= -3: result='A' 
    elif (df.투약후[x] - df.투약전[x]) <= 0: result='B' 
    else: result='C'
    lst.append(result)
df['등급'] = lst

print(df[df['등급']=='A'].ID.count())

121


In [9]:
## answer
df['dels'] = df['투약후'] - df['투약전']

def groupby_filter(x):
    if x <=-3:
        return 'A'
    
    elif x <=0:
        return 'B'
    
    else:
        return 'C'
    
df['groupby'] = df['dels'].apply(groupby_filter)
result = df['groupby'].value_counts().sort_index()['A']
print(result)

121


In [10]:
df.head()

,ID,투약전,투약후,등급,dels,groupby
0,user_1,55.137,56.106,C,0.969,C
1,user_2,66.584,60.409,A,-6.175,A
2,user_3,52.259,50.639,B,-1.620,B
3,user_4,77.081,69.164,A,-7.917,A
4,user_5,62.677,57.622,A,-5.055,A


### 3-1-b
카이제곱검정 통계량을 반올림하여 소숫점 이하 3째자리까지 구하여라

In [21]:
cnt = df['등급'].value_counts().sort_index().to_frame('count')
cnt['expected'] = [cnt['count'].sum()*0.5, cnt['count'].sum()*0.25, cnt['count'].sum()*0.25]

from scipy.stats import chisquare
statistics, pvalue = chisquare(cnt['count'], cnt['expected'])
print(round(statistics, 3))

3.613


In [18]:
## answer
target = df['groupby'].value_counts().sort_index().to_frame('groupby')
target['expected'] = [target['groupby'].sum()*0.5,target['groupby'].sum()*0.25,target['groupby'].sum()*0.25]

from scipy.stats import chisquare
s,p = chisquare(target['groupby'],target['expected'])

round_s = round(s,3)
print(round_s)

3.613


### 3-1-c
카이제곱 검정 p값을 반올림하여 소숫점 이하 3자리까지 구하고, 유의수준 0.05하에서 귀무가설과 대립가설중 유의한 가설을 하나를 선택하시오(귀무/대립)

In [22]:
print(round(pvalue,3))
if round(pvalue,3) < 0.05:
    print('대립')
else:
    print('귀무')

0.164
귀무


A,B 공장에서 생산한 기계들의 rpm 값들을 기록한 데이터이다. <br>
대응 표본 t 검정을 통해 B공장 제품들이 A 공장 제품들보다 rpm이 높다고 말할 수 있는지 검정하려 한다
### 3-2-a
A,B 공장 각각 정규성을 가지는지 샤피로 검정을 통해 확인하라. (각 공장의 pvalue 출력할 것)

In [23]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p3_2_.csv')
df.head()

,rpm,group
0,163.6,A
1,246.9,A
2,234.9,A
3,156.9,A
4,226.6,A


In [25]:
from scipy.stats import shapiro
s, pA = shapiro(df[df['group']=='A'].rpm)
s, pB = shapiro(df[df['group']=='B'].rpm)
print(pA, pB)
## 두 공장 다 정규성을 갖는다

0.397915244102478 0.9562925100326538


### 3-2-b
대응 표본 t 검정을 통해 B공장 제품들의 rpm이 A 공장 제품의 rpm보다 크다고 말할 수 있는지 검정하라. pvalue를 소숫점 이하 3자리까지 출력하고 귀무가설, 대립가설 중 하나를 출력하라

In [37]:
from scipy.stats import ttest_rel
_, pvalue = ttest_rel(df[df['group']=='B'].rpm, df[df['group']=='A'].rpm, alternative='greater')
print(round(pvalue, 3))
print('대립')

0.009
대립
